In [2]:
import numpy as np
import torch


In [3]:
# initialise all necesarry functions here. Maybe should be moved to another file and import later
def one_hot(idx, length):
    idx = np.array(idx)
    x = np.zeros([len(idx), length])
    x[np.arange(len(idx)), idx] = 1.0
    return x

# Step 1: Extract from Matrix to build a global graph

In [4]:
# Question: How to extract data from matrix and build it into a graph?

# Step 2: Build a global graph

In [5]:
# Prepare variables 

# (sample)
global_ids = [0,1,1,1,1,2,3,3,3,3,4,5,5,5,5] # assume 3 matrices: 0 = user; 2 = item; 4 = description

# encode it
global_encoded = one_hot(global_ids, max(global_ids)+1)
global_encoded

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [6]:
# Question: How to create the relationship into the following format?

In [9]:
# parent = [target parent ids]
# children = [target child index, relationship weight]

# user
user_parent = [[2,2],[2,2],[],[]]
user_children = [\
                 [[0,4],[1,3]],\
                 [[2,1],[3,4]],\
                 [],\
                 []\
                ]
# item
item_parent = [[0,4],[0],[0,4],[0]]
item_children =[\
                [[0,4],[0,1]],\
                [[1,3]],\
                [[2,1],[2,1]],\
                [[3,4]]\
               ]

# feature
feature_parent = [[2],[2],[],[]]
feature_children = [[[0,1]],[[2,1]],[],[]]

data = [user_parent, user_children, item_parent, item_children, feature_parent, feature_children]
data

[[[2, 2], [2, 2], [], []],
 [[[0, 4], [1, 3]], [[2, 1], [3, 4]], [], []],
 [[0, 4], [0], [0, 4], [0]],
 [[[0, 4], [0, 1]], [[1, 3]], [[2, 1], [2, 1]], [[3, 4]]],
 [[2], [2], [], []],
 [[[0, 1]], [[2, 1]], [], []]]

In [ ]:
# Question: How to merge similar relationship to make it undirected?

# Step 3: Extract local enclosing subgraph


In [13]:
def get_parent_id(data, parent_id):
    all_parents_pos = [id for (id,item) in enumerate(data) if item % 2 == 0]    
    print('function', all_parents_pos)
    local_parent_id = int(parent_id/2) # give index 0,1,2 etc. from parent id 2,4,6 etc.
    parent_idx = all_parents_pos[local_parent_id]
    return parent_idx

def get_relationships(idx):
    # variables
    # idx = index
    # pos = position

    c_idx = idx
    target_parents = []
    target_children = []

    all_parents_pos = [id for (id,item) in enumerate(global_ids) if item % 2 == 0]    

    # calculate positions
    c_pos = global_ids[idx] # child position
    p_id = c_pos if(c_idx in all_parents_pos) else c_pos - 1
    local_parent_id = int(p_id/2) # give index 0,1,2 etc. from parent id 2,4,6 etc.
    p_idx = all_parents_pos[local_parent_id]
    cp_pos = c_idx - p_idx - 1

    # return
    target_parents = data[p_idx][cp_pos] if(not p_idx == c_idx ) else data[p_id]
    target_children = data[c_pos][cp_pos] if(not p_idx == c_idx ) else data[p_id+1]

    return [target_parents,target_children]

In [17]:
# Test
print('Node labels', global_ids)

index = 7
relationships = get_relationships(index)
print('index:', index)
print('target parent:', relationships[0])
print('target weights:', relationships[1])

Node labels [0, 1, 1, 1, 1, 2, 3, 3, 3, 3, 4, 5, 5, 5, 5]
index: 7
target parent: [[2, 1]]
target weights: [[1, 3]]


# Step 4: Convert to bipartite graph

In [14]:
# Question: How to convert it to PyTorch, for training later?

# Step 5: Train GNN